<a href="https://colab.research.google.com/github/UmairPirzada/PIAIC_Q1-Q2_Agentic-AI/blob/main/Function_calling_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2"

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_GENAI_KEY"))

## Set up a model with tools

This example uses 3 functions that control a simple hypothetical lighting system. Using these functions requires them to be called in a specific order. For example, you must turn the light system on before you can change color.

While you can pass these directly to the model and let it try to call them correctly, specifying the `function_calling_config` gives you precise control over the functions that are available to the model.

In [ ]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

Create a helper function for setting `function_calling_config` on `tool_config`.

In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

## Text-only mode: `NONE`

If you have provided the model with tools, but do not want to use those tools for the current conversational turn, then specify `NONE` as the mode. `NONE` tells the model not to make any function calls, and will behave as though none have been provided.

In [ ]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2001.09ms


I can turn lights on and off, and I can set the color of the lights.



## Automatic mode: `AUTO`

To allow the model to decide whether to respond in text or call specific functions, you can specify `AUTO` as the mode.

In [ ]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4855.21ms


function_call {
  name: "enable_lights"
  args {
  }
}



In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "enable_lights",
                  "args": {}
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -1.986752901454262e-07
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 171,
        "candidates_token_count": 3,
        "total_token_count": 174
      },
      "model_version": "gemini-1.5-pro-002"
    }),
)

In [ ]:
response.parts[0]

function_call {
  name: "enable_lights"
  args {
  }
}

## Function-calling mode: `ANY`

Setting the mode to `ANY` will force the model to make a function call. By setting `allowed_function_names`, the model will only choose from those functions. If it is not set, all of the functions in `tools` are candidates for function calling.

In this example system, if the lights are already on, then the user can change color or turn the lights off.

In [ ]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts[0])

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1771.23ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 785.90ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 785.13ms


function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "FF00FF"
      }
    }
  }
}



In [ ]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_color",
                  "args": {
                    "rgb_hex": "FF00FF"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.00021397739571208754
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 172,
        "candidates_token_count": 12,
        "total_token_count": 184
      },
      "model_version": "gemini-1.5-pro-002"
    }),
)


In [ ]:
print(response.parts)

[function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "FF00FF"
      }
    }
  }
}
]


In [ ]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place Blue!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "0000FF"
      }
    }
  }
}



## Automatic function calling

`tool_config` works when enabling automatic function calling too.

In [ ]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)

LIGHTBOT: Lights enabled.


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Lights enabled."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.525895873705546
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 169,
        "candidates_token_count": 3,
        "total_token_count": 172
      },
      "model_version": "gemini-1.5-pro-002"
    }),
)

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_color",
                  "args": {
                    "rgb_hex": "0000FF"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.007631147710176615
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 201,
        "candidates_token_count": 13,
        "total_token_count": 214
      },
      "model_version": "gemini-1.5-pro-002"
    }),
)

In [ ]:
def set_light_values(brightness, color_temp):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

In [ ]:
from typing import Literal

In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [ ]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1948.31ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1466.43ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1013.47ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 685.66ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1088.14ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 658.68ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1350.06ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:ge

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [ ]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Ok, I've turned on the disco ball, started playing "Never gonna give you up" at 120bpm, and dimmed the lights to 50% brightness.  Let's get this party started!



In [ ]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                             tools=[multiply])

model._tools.to_proto()

[function_declarations {
   name: "multiply"
   description: "returns a * b."
   parameters {
     type_: OBJECT
     properties {
       key: "b"
       value {
         type_: NUMBER
       }
     }
     properties {
       key: "a"
       value {
         type_: NUMBER
       }
     }
     required: "a"
     required: "b"
   }
 }]

In [ ]:
model._tools.to_proto()

[function_declarations {
   name: "multiply"
   description: "Returns the product of two numbers."
   parameters {
     type_: OBJECT
     properties {
       key: "b"
       value {
         type_: NUMBER
       }
     }
     properties {
       key: "a"
       value {
         type_: NUMBER
       }
     }
     required: "a"
     required: "b"
   }
 }]

In [ ]:
calculator = genai.protos.Tool(
    function_declarations=[
      genai.protos.FunctionDeclaration(
        name='multiply',
        description="Returns the product of two numbers.",
        parameters=genai.protos.Schema(
            type=genai.protos.Type.OBJECT,
            properties={
                'a':genai.protos.Schema(type=genai.protos.Type.NUMBER),
                'b':genai.protos.Schema(type=genai.protos.Type.NUMBER)
            },
            required=['a','b']
        )
      )
    ])

In [ ]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]}

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash', tools=calculator)
chat = model.start_chat()

response = chat.send_message(
    f"What's 234551 X 325552 ?",
)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2226.63ms


In [ ]:
response.candidates

[content {
  parts {
    function_call {
      name: "multiply"
      args {
        fields {
          key: "b"
          value {
            number_value: 325552
          }
        }
        fields {
          key: "a"
          value {
            number_value: 234551
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.00061810257223745191
]

In [ ]:
fc = response.candidates[0].content.parts[0].function_call
assert fc.name == 'multiply'

result = fc.args['a'] * fc.args['b']
result

76358547152.0

In [ ]:
response = chat.send_message(
    genai.protos.Content(
    parts=[genai.protos.Part(
        function_response = genai.protos.FunctionResponse(
          name='multiply',
          response={'result': result}))]))

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "234551 multiplied by 325552 is 76,358,547,152.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.025572675647157612
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 121,
        "candidates_token_count": 33,
        "total_token_count": 154
      },
      "model_version": "gemini-1.5-flash"
    }),
)